In [ ]:
!pip --default-timeout=1000 install mindspore-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 MB 3.7 MB/s eta 0:00:00


In [ ]:
import mindspore
from mindspore import nn, ops
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset

[WARNING] ME(448:139721996076864,MainProcess):2023-04-12-23:40:16.954.250 [mindspore/run_check/_check_version.py:79] MindSpore version 1.10.0 and cuda version 11.8.89 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.


# Processing a Dataset



MindSpore provides Pipeline-based [Data Engine](https://www.mindspore.cn/docs/zh-CN/master/design/data_engine.html) and achieves efficient data preprocessing through [Dataset](https://www.mindspore.cn/tutorials/en/master/beginner/dataset.html) and [Transforms](https://www.mindspore.cn/tutorials/en/master/beginner/transforms.html). In this tutorial, we use the Mnist dataset and pre-process dataset by using the data transformations provided by `mindspore.dataset`, after automatically downloaded.


In [ ]:
!wget https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/datasets/MNIST_Data.zip
!unzip MNIST_Data.zip

--2023-04-12 23:40:19--  https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/datasets/MNIST_Data.zip
Resolving mindspore-website.obs.cn-north-4.myhuaweicloud.com (mindspore-website.obs.cn-north-4.myhuaweicloud.com)... 49.4.112.92, 49.4.112.91, 121.36.121.84
Connecting to mindspore-website.obs.cn-north-4.myhuaweicloud.com (mindspore-website.obs.cn-north-4.myhuaweicloud.com)|49.4.112.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10754903 (10M) [application/zip]
Saving to: ‘MNIST_Data.zip’

MNIST_Data.zip      100%[===================>]  10.26M  6.37MB/s    in 1.6s    

2023-04-12 23:40:22 (6.37 MB/s) - ‘MNIST_Data.zip’ saved [10754903/10754903]

Archive:  MNIST_Data.zip
   creating: MNIST_Data/test/
  inflating: MNIST_Data/test/t10k-images-idx3-ubyte  
  inflating: MNIST_Data/test/t10k-labels-idx1-ubyte  
   creating: MNIST_Data/train/
  inflating: MNIST_Data/train/train-images-idx3-ubyte  
  inflating: MNIST_Data/train/train-labels-idx1-uby

In [ ]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

print(train_dataset.get_col_names())

['image', 'label']


In [ ]:
test_dataset

In [ ]:
test_dataset.create_tuple_iterator()

Dataset in MindSpore uses the Data Processing Pipeline, which requires specifying operations such as map, batch, and shuffle. Here we use `map` to transform the image data and the label, and then pack the processed dataset into a batch of size 64.

In [ ]:
def datapipe(dataset, batch_size):
  image_transforms = [
    vision.Rescale(1.0 / 255.0, 0),
    vision.Normalize(mean=(0.1307,), std=(0.3081,)),
    vision.HWC2CHW(),
  ]

  label_transform = transforms.TypeCast(mindspore.int32)

  dataset = dataset.map(image_transforms, 'image')
  dataset = dataset.map(label_transform, 'label')
  dataset = dataset.batch(batch_size)
  return dataset

train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(train_dataset, 64)

In [ ]:
test_dataset.get_col_names()

['image', 'label']

In [ ]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

RuntimeError: ignored

# Building Network

`mindspore.nn` class is the base class for building all networks and is the basic unit of the network. When the user needs to customize the network, you can inherit the `nn.Cell` class and override the `__init__` method and the `construct` method. `__init__` contains the definitions of all network layers, and `construct` contains the transformation process of the data ([Tensor](https://www.mindspore.cn/tutorials/en/master/beginner/tensor.html)) (i.e. the construction process of the [computational graph](https://www.mindspore.cn/tutorials/en/master/advanced/compute_graph.html)).

In [ ]:
# Define model
class Network(nn.Cell):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.dense_relu_sequential = nn.SequentialCell(
      nn.Dense(28*28, 512),
      nn.ReLU(),
      nn.Dense(512, 512),
      nn.ReLU(),
      nn.Dense(512, 10)
    )

  def construct(self, x):
    x = self.flatten(x)
    logits = self.dense_relu_sequential(x)
    return logits

model = Network()
print(model)

# Training Model

In [ ]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), 1e-2)

In model training, a complete training process (step) requires the following three steps:

1. **Forward calculation**: model predicts results (logits) and finds the prediction loss (loss) with the correct label (label).
2. **Backpropagation**: Using an automatic differentiation mechanism, the gradients of the model parameters (parameters) with respect to the loss are automatically found.
3. **Parameter optimization**: update the gradient to the parameter.

MindSpore uses a functional automatic differentiation mechanism, implemented through the steps above:

1. Define forward calculation function.
2. Obtain the gradient calculation function by function transformation.
3. Define training functions, and perform forward computation, back propagation and parameter optimization.

In [ ]:
# Define forward function
def forward_fn(data, label):
  logits = model(data)
  loss = loss_fn(logits, label)
  return loss, logits

# Get gradient function
grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)
#grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

# Define function of one-step training
def train_step(data, label):
  (loss, _), grads = grad_fn(data, label)
  optimizer(grads)
  return loss

def train(model, dataset):
  size = dataset.get_dataset_size()
  model.set_train()
  for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
    loss = train_step(data, label)
    if batch % 100 == 0:
      loss, current = loss.asnumpy(), batch
      print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

In addition to training, we define test functions that are used to evaluate the performance of the model.



In [ ]:
def test(model, dataset, loss_fn):
  num_batches = dataset.get_dataset_size()
  model.set_train(False)
  total, test_loss, correct = 0, 0, 0
  for data, label in dataset.create_tuple_iterator():
    pred = model(data)
    total += len(data)
    test_loss += loss_fn(pred, label).asnumpy()
    correct += (pred.argmax(1) == label).asnumpy().sum()
  test_loss /= num_batches
  correct /= total
  print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process requires several iterations of the dataset, and one complete iteration is called an epoch. In each round, the training set is traversed for training and the test set is used for prediction at the end. The loss value and prediction accuracy (Accuracy) of each round are printed, and it can be seen that the loss is decreasing and Accuracy is increasing.

In [ ]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset)
    test(model, test_dataset, loss_fn)
print("Done!")